# Entrenamiento Final - YOLOv11 Pothole Detection

Este notebook realiza el entrenamiento final del modelo YOLOv11x-seg usando los mejores hiperparámetros encontrados en la optimización bayesiana.

## Hardware: RTX 5090 (32GB VRAM)
## Model: YOLOv11x-seg (Instance Segmentation)

In [ ]:
import os
import yaml
import json
import torch
from pathlib import Path
from datetime import datetime
from ultralytics import YOLO
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from PIL import Image
import cv2

# Configuración de estilo
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (14, 10)

In [ ]:
# Verificar GPU disponible
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM Total: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    print(f"PyTorch Version: {torch.__version__}")
    print(f"CUDA Version: {torch.version.cuda}")

In [ ]:
# Configuración de rutas
BASE_DIR = Path('..')
CONFIG_DIR = BASE_DIR / 'config'
MODELS_DIR = BASE_DIR / 'models'
OPTIMIZATION_DIR = MODELS_DIR / 'optimization'

# Cargar configuración del dataset
DATASET_CONFIG = CONFIG_DIR / 'pothole_dataset.yaml'

print(f"Dataset Config: {DATASET_CONFIG}")
print(f"Models Directory: {MODELS_DIR}")

## Cargar Mejores Hiperparámetros

In [ ]:
# Cargar mejores hiperparámetros de la optimización bayesiana
best_params_file = OPTIMIZATION_DIR / 'best_hyperparameters.json'

if best_params_file.exists():
    with open(best_params_file, 'r') as f:
        optimization_results = json.load(f)
    
    best_params = optimization_results['best_params']
    best_value = optimization_results['best_value']
    
    print("MEJORES HIPERPARÁMETROS CARGADOS:")
    print("="*80)
    print(f"mAP50-95 en optimización: {best_value:.4f}")
    print(f"\nHiperparámetros:")
    for param, value in best_params.items():
        print(f"  {param}: {value}")
else:
    print("ADVERTENCIA: No se encontró archivo de mejores hiperparámetros.")
    print("Usando hiperparámetros por defecto.")
    best_params = {}

## Configuración de Entrenamiento Final

In [ ]:
# Configuración de entrenamiento
TRAINING_CONFIG = {
    'model': 'yolo11x-seg.pt',  # Modelo más grande para mejor accuracy
    'epochs': 300,
    'patience': 50,
    'device': 0,
    'workers': 8,
    'project': str(MODELS_DIR / 'final_training'),
    'name': 'yolo11x_pothole_final',
    'exist_ok': True,
    'pretrained': True,
    'verbose': True,
    'plots': True,
    'save': True,
    'save_period': 10,  # Guardar checkpoint cada 10 epochs
    'val': True,
    'resume': False,  # Cambiar a True para continuar entrenamiento
    'amp': True,  # Automatic Mixed Precision para mejor uso de VRAM
    'fraction': 1.0,  # Usar todo el dataset
    'profile': False,
    'freeze': None,  # No congelar capas
    'multi_scale': False,  # Desactivar multi-scale para mayor velocidad
    'overlap_mask': True,  # Permitir overlap en máscaras
    'mask_ratio': 4,  # Ratio de downsampling para máscaras
    'dropout': 0.0,  # Sin dropout
    'val_split': 0.0,  # No usar split automático (ya tenemos valid/)
}

# Combinar con mejores hiperparámetros
training_params = {**TRAINING_CONFIG, **best_params}

print("\nCONFIGURACIÓN DE ENTRENAMIENTO FINAL:")
print("="*80)
for key, value in training_params.items():
    print(f"  {key}: {value}")

## Inicializar Modelo

In [ ]:
# Cargar modelo YOLOv11x-seg
print(f"\nCargando modelo: {TRAINING_CONFIG['model']}")
model = YOLO(TRAINING_CONFIG['model'])

# Mostrar información del modelo
print(f"\nModelo cargado exitosamente!")
print(f"Tarea: {model.task}")
print(f"Modelo: {model.model_name}")

## Entrenar Modelo

Este proceso puede tomar varias horas dependiendo del tamaño del dataset y la configuración.

In [ ]:
# Iniciar entrenamiento
print("\n" + "="*80)
print("INICIANDO ENTRENAMIENTO FINAL")
print("="*80)
print(f"Inicio: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\nEsto puede tomar varias horas...\n")

start_time = datetime.now()

# Entrenar
results = model.train(
    data=str(DATASET_CONFIG),
    **training_params
)

end_time = datetime.now()
training_duration = end_time - start_time

print("\n" + "="*80)
print("ENTRENAMIENTO COMPLETADO")
print("="*80)
print(f"Fin: {end_time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Duración total: {training_duration}")
print(f"\nModelo guardado en: {results.save_dir}")

## Evaluación del Modelo

In [ ]:
# Validar modelo en el conjunto de validación
print("\nEvaluando modelo en conjunto de validación...")
metrics = model.val()

print("\nMÉTRICAS DE VALIDACIÓN:")
print("="*80)
print(f"Box mAP50: {metrics.box.map50:.4f}")
print(f"Box mAP50-95: {metrics.box.map:.4f}")
print(f"Mask mAP50: {metrics.seg.map50:.4f}")
print(f"Mask mAP50-95: {metrics.seg.map:.4f}")
print(f"\nPrecision (Box): {metrics.box.mp:.4f}")
print(f"Recall (Box): {metrics.box.mr:.4f}")
print(f"Precision (Mask): {metrics.seg.mp:.4f}")
print(f"Recall (Mask): {metrics.seg.mr:.4f}")

In [ ]:
# Guardar métricas finales
final_metrics = {
    'training_duration': str(training_duration),
    'final_epoch': results.epoch,
    'box_map50': float(metrics.box.map50),
    'box_map50_95': float(metrics.box.map),
    'mask_map50': float(metrics.seg.map50),
    'mask_map50_95': float(metrics.seg.map),
    'box_precision': float(metrics.box.mp),
    'box_recall': float(metrics.box.mr),
    'mask_precision': float(metrics.seg.mp),
    'mask_recall': float(metrics.seg.mr),
    'training_config': training_params,
    'timestamp': datetime.now().isoformat()
}

metrics_file = Path(results.save_dir) / 'final_metrics.json'
with open(metrics_file, 'w') as f:
    json.dump(final_metrics, f, indent=2)

print(f"\nMétricas guardadas en: {metrics_file}")

## Visualización de Resultados de Entrenamiento

In [ ]:
# Cargar resultados de entrenamiento
results_csv = Path(results.save_dir) / 'results.csv'
if results_csv.exists():
    df_results = pd.read_csv(results_csv)
    df_results.columns = df_results.columns.str.strip()
    
    print("Columnas disponibles en results.csv:")
    print(df_results.columns.tolist())
    
    # Gráficas de métricas de entrenamiento
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # mAP50 y mAP50-95
    axes[0, 0].plot(df_results['epoch'], df_results['metrics/mAP50(B)'], label='Box mAP50', linewidth=2)
    axes[0, 0].plot(df_results['epoch'], df_results['metrics/mAP50-95(B)'], label='Box mAP50-95', linewidth=2)
    axes[0, 0].plot(df_results['epoch'], df_results['metrics/mAP50(M)'], label='Mask mAP50', linewidth=2, linestyle='--')
    axes[0, 0].plot(df_results['epoch'], df_results['metrics/mAP50-95(M)'], label='Mask mAP50-95', linewidth=2, linestyle='--')
    axes[0, 0].set_xlabel('Epoch', fontsize=12)
    axes[0, 0].set_ylabel('mAP', fontsize=12)
    axes[0, 0].set_title('Mean Average Precision', fontsize=14, fontweight='bold')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Precision y Recall
    axes[0, 1].plot(df_results['epoch'], df_results['metrics/precision(B)'], label='Box Precision', linewidth=2)
    axes[0, 1].plot(df_results['epoch'], df_results['metrics/recall(B)'], label='Box Recall', linewidth=2)
    axes[0, 1].set_xlabel('Epoch', fontsize=12)
    axes[0, 1].set_ylabel('Score', fontsize=12)
    axes[0, 1].set_title('Precision & Recall', fontsize=14, fontweight='bold')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Losses
    loss_cols = [col for col in df_results.columns if 'loss' in col.lower() and 'train' in col.lower()]
    for col in loss_cols:
        axes[1, 0].plot(df_results['epoch'], df_results[col], label=col.replace('train/', ''), linewidth=2)
    axes[1, 0].set_xlabel('Epoch', fontsize=12)
    axes[1, 0].set_ylabel('Loss', fontsize=12)
    axes[1, 0].set_title('Training Losses', fontsize=14, fontweight='bold')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Learning Rate
    if 'lr/pg0' in df_results.columns:
        axes[1, 1].plot(df_results['epoch'], df_results['lr/pg0'], linewidth=2, color='red')
        axes[1, 1].set_xlabel('Epoch', fontsize=12)
        axes[1, 1].set_ylabel('Learning Rate', fontsize=12)
        axes[1, 1].set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
        axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(Path(results.save_dir) / 'training_metrics.png', dpi=300, bbox_inches='tight')
    plt.show()
else:
    print("No se encontró archivo results.csv")

## Exportar Modelo para Inferencia

In [ ]:
# Exportar a diferentes formatos
export_dir = MODELS_DIR / 'exports'
export_dir.mkdir(exist_ok=True)

print("Exportando modelo a diferentes formatos...\n")

# Copiar el mejor modelo a directorio de exportación
best_model_path = Path(results.save_dir) / 'weights' / 'best.pt'
final_model_path = export_dir / 'yolo11x_pothole_best.pt'

import shutil
if best_model_path.exists():
    shutil.copy(best_model_path, final_model_path)
    print(f"✓ Modelo PyTorch guardado: {final_model_path}")

# Cargar el mejor modelo para exportación
best_model = YOLO(str(best_model_path))

# Exportar a ONNX (para compatibilidad multiplataforma)
try:
    onnx_path = best_model.export(format='onnx', dynamic=True, simplify=True)
    print(f"✓ Modelo ONNX exportado: {onnx_path}")
except Exception as e:
    print(f"✗ Error exportando ONNX: {e}")

# Exportar a TorchScript (para producción PyTorch)
try:
    torchscript_path = best_model.export(format='torchscript')
    print(f"✓ Modelo TorchScript exportado: {torchscript_path}")
except Exception as e:
    print(f"✗ Error exportando TorchScript: {e}")

print(f"\nTodos los modelos exportados a: {export_dir}")

## Pruebas de Inferencia

In [ ]:
# Cargar modelo final para pruebas
final_model = YOLO(str(final_model_path))

print("Modelo cargado y listo para inferencia!")
print(f"\nPara usar el modelo en inferencia:")
print(f"  model = YOLO('{final_model_path}')")
print(f"  results = model.predict(source='image.jpg', conf=0.25, iou=0.7)")

In [ ]:
# Función de prueba de inferencia en imágenes del conjunto de validación
from pathlib import Path
import random

# Buscar imágenes de validación
valid_images_dir = Path('../dataset/valid/images')
if valid_images_dir.exists():
    valid_images = list(valid_images_dir.glob('*.png')) + list(valid_images_dir.glob('*.jpg'))
    
    if len(valid_images) > 0:
        # Seleccionar 5 imágenes aleatorias
        sample_images = random.sample(valid_images, min(5, len(valid_images)))
        
        print(f"Ejecutando inferencia en {len(sample_images)} imágenes de muestra...\n")
        
        # Ejecutar predicciones
        results = final_model.predict(
            source=sample_images,
            conf=0.25,
            iou=0.7,
            save=True,
            project=str(MODELS_DIR / 'inference_samples'),
            name='validation_samples',
            exist_ok=True
        )
        
        print(f"\nResultados de inferencia guardados en: {MODELS_DIR / 'inference_samples' / 'validation_samples'}")
        
        # Mostrar estadísticas
        total_detections = sum([len(r.boxes) for r in results])
        print(f"\nEstadísticas:")
        print(f"  Total de imágenes: {len(sample_images)}")
        print(f"  Total de baches detectados: {total_detections}")
        print(f"  Promedio de baches por imagen: {total_detections/len(sample_images):.2f}")
    else:
        print("No se encontraron imágenes en el directorio de validación.")
else:
    print(f"Directorio de validación no encontrado: {valid_images_dir}")
    print("Por favor, actualiza la ruta del dataset en el notebook.")

## Resumen Final

In [ ]:
print("\n" + "="*80)
print("RESUMEN DE ENTRENAMIENTO FINAL")
print("="*80)
print(f"\nModelo: YOLOv11x-seg")
print(f"Duración de entrenamiento: {training_duration}")
print(f"Epochs completados: {results.epoch}")
print(f"\nMétricas Finales:")
print(f"  Box mAP50-95: {metrics.box.map:.4f}")
print(f"  Mask mAP50-95: {metrics.seg.map:.4f}")
print(f"  Precision (Mask): {metrics.seg.mp:.4f}")
print(f"  Recall (Mask): {metrics.seg.mr:.4f}")
print(f"\nModelo final guardado en:")
print(f"  PyTorch: {final_model_path}")
print(f"  Directorio de exportación: {export_dir}")
print(f"\nArchivos generados:")
print(f"  Métricas: {metrics_file}")
print(f"  Resultados: {results.save_dir}")
print(f"\n{'='*80}")
print("¡Entrenamiento completado exitosamente!")
print("Siguiente paso: Clasificación según norma ASTM D6433-03")
print("="*80)